In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
from src.sequential_dataset import JointGraphDataModule, SequentialLegoDataJointGraph

ds = SequentialLegoDataJointGraph("../data/", 'gen', 300)


In [3]:
import matplotlib.pyplot as plt

def plot_voxels(voxels, ax=None, **kwargs):
    if ax is None:
        ax = plt.figure().add_subplot(projection='3d')
    ax.voxels(voxels, edgecolor='k', **kwargs)
    print(voxels.shape)
    plt.show()

In [4]:
from src.models.joint_graph_model import LegoNet

model = LegoNet.load_from_checkpoint("../lightning_logs/version_21/checkpoints/epoch=0-step=18.ckpt")

In [5]:
import torch
def get_attn_brick_to_point(lego, net, brick_idx):
    _, attns = net.process_graph(lego, return_attn=True)
    attn = attns[0] #TODO: take average

    edge_index, weight = attn
    edge_mask = edge_index[0] == brick_idx

    point_weights = weight[edge_mask].reshape(-1)
    points = edge_index[1][edge_mask].reshape(-1)

    point_attn = torch.zeros(len(lego['point'].pos))
    point_attn[points] = point_weights

    return point_attn.detach()

In [7]:
%matplotlib
import torch

lego = ds[120]
points = lego['point'].pos
focus_brick = 10

fig = plt.figure()
ax = fig.add_subplot(121, projection='3d')

attn = get_attn_brick_to_point(lego, model, focus_brick)
ax.scatter(points[:,0], points[:,1], points[:,2], 'bo-', c=attn)

from src.lego_model import LegoModel

focus_idxs = torch.Tensor([focus_brick]).to(int)
voxels, voxels_collisions = LegoModel.to_voxels(lego['lego'], 1, 1, 1.2, focus=focus_idxs)

ax2 = fig.add_subplot(122, projection='3d')
plot_voxels(voxels, ax2, alpha=0.2)
plot_voxels(voxels_collisions, ax2, facecolors='r')
plt.show()

Using matplotlib backend: MacOSX
torch.Size([14, 14, 4])
torch.Size([14, 14, 4])
